In [ ]:
import cv2

img = cv2.imread("input.jpg")


In [ ]:
import pytesseract
from PIL import Image

# If tesseract is not in PATH, set it manually
# pytesseract.pytesseract.tesseract_cmd = r"/usr/bin/tesseract"

image_path = "/home/kawsar/Desktop/Image-Masking/POC-table-masking/src/images/ss-1.jpeg"
image_path = "/home/kawsar/Downloads/selected_sap_images/9325_screenshot.png"

img = Image.open(image_path)

text = pytesseract.image_to_string(img, lang="eng", config="--psm 12")
# remove empty lines
text = "\n".join([line for line in text.split("\n") if line.strip() != ""])

print(text)


In [ ]:
import cv2
import pytesseract

# image_path = "image.png"

img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Optional preprocessing
gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

text = pytesseract.image_to_string(gray, lang="eng", config="--psm 12")
text = "\n".join([line for line in text.split("\n") if line.strip() != ""])

print(text)


In [22]:
import cv2
import pytesseract
from pytesseract import Output

img = cv2.imread(image_path)

data = pytesseract.image_to_data(
img,
lang="eng",
config="--oem 3 --psm 12",
output_type=pytesseract.Output.DICT
)

lines = {}

# -----------------------------
# Group words into lines
# -----------------------------
for i in range(len(data["text"])):
    text = data["text"][i].strip()
    conf = int(data["conf"][i])

    # level 5 = word
    if not text or conf <= 0 or data["level"][i] != 5:
        continue

    key = (
        data["block_num"][i],
        data["par_num"][i],
        data["line_num"][i]
    )

    lines.setdefault(key, []).append(i)

line_texts = []

# -----------------------------
# Build line-level text + box
# -----------------------------
for indices in lines.values():
    texts = [data["text"][i] for i in indices]

    x_min = min(data["left"][i] for i in indices)
    y_min = min(data["top"][i] for i in indices)
    x_max = max(data["left"][i] + data["width"][i] for i in indices)
    y_max = max(data["top"][i] + data["height"][i] for i in indices)

    line_texts.append({
        "text": " ".join(texts),
        "box": [
            [x_min, y_min],
            [x_max, y_min],
            [x_max, y_max],
            [x_min, y_max],
        ],
        "prob": min(int(data["conf"][i]) for i in indices) / 100.0
    })

# -----------------------------
# Sort lines top → bottom
# -----------------------------
line_texts.sort(key=lambda x: x["box"][0][1])


In [23]:
for line in line_texts:
    print(line["text"], line["box"], line["prob"])

Wh) [[757, 8], [772, 8], [772, 18], [757, 18]] 0.33
ket [[628, 8], [709, 8], [709, 21], [628, 21]] 0.1
© [[1054, 40], [1063, 40], [1063, 48], [1054, 48]] 0.59
LD [[1224, 42], [1241, 42], [1241, 53], [1224, 53]] 0.96
rie [[164, 43], [233, 43], [233, 54], [164, 54]] 0.27
New Purchase Order [[33, 103], [214, 103], [214, 118], [33, 118]] 0.96
General Information [[36, 177], [157, 177], [157, 187], [36, 187]] 0.94
Items [[191, 177], [223, 177], [223, 187], [191, 187]] 0.55
Limit Items [[257, 177], [323, 177], [323, 187], [257, 187]] 0.76
Notes [[681, 177], [715, 177], [715, 187], [681, 187]] 0.88
Attachments [[748, 177], [824, 177], [824, 187], [748, 187]] 0.91
Delivery and Invoice [[356, 177], [480, 177], [480, 190], [356, 190]] 0.96
Supplier Contact Data [[513, 177], [647, 177], [647, 190], [513, 190]] 0.94
Approval Details [[857, 177], [957, 177], [957, 190], [857, 190]] 0.93
Basic Data [[33, 213], [101, 213], [101, 223], [33, 223]] 0.96
Organization [[648, 213], [732, 213], [732, 226], 

In [24]:
# write them into json file
import json
with open("output.json", "w") as f:
    json.dump(line_texts, f, indent=4)